# SOLUCIÓN PARA EL RETO "NOSOTROS"

### DESCRIPCIÓN:

Este notebook de Jupyter contiene la solución para el reto de la hackathon. Su funcionamiento se basa en la interacción con las APIs de Instagram y Twitter (o sus réplicas para este evento).

Al introducir un archivo JSON con diferentes hashtags repartidos en categorías, el notebook es capaz de obtener las publicaciones más relevantes para la categoría escogida consultando los hashtags que contiene. 

Una vez obtenidas las publicaciones se realiza un cribado en función del impacto de los usuarios dentro de la categoría indicada, según el siguiente criterio:

>**Instagram:** Se calcula el impacto de los usuarios obteniendo el número de publicaciones relevantes dentro de una categoría determinada, cribando aquellas que no entran dentro de las que tienen más comentarios y likes (en ese orden).

>**Twitter:** Se calcula el índice de impacto de un usuario mediante la suma de las respuestas, likes y retweets de los tweets que utilizan los hashtags de la categoría.

#### Comentarios adicionales:

Las categorías de ejemplo que se han utilizado son:
*'hashtags_fitness',
'hashtags_food',
'hashtags_travel'*

Se incluye en cada función principal la medida del tiempo de ejecución.

#### Librerías utilizadas:

In [1]:
import requests
import json
import pandas as pd
import time

## FUNCIONES PARA INSTAGRAM

#### Función principal:

In [2]:
def main_instagram():
    start = time.time()
    
    #Credenciales y variables:
    insta_user = '17473'
    insta_key = '2jkcsHxjpcYDwtlj2idP'
    hashtags_category = 'hashtags_travel'
    target_users = 20
    user_id = insta_user
    #Tipo de contenido: 'recent' o 'top'
    mediaType = 'recent'
    
    #Obtención del ránking de usuarios:
    best_posts = get_best_media_posts(hashtags_category, user_id, mediaType)
    best_posts = best_posts.reset_index(drop=True)
    
    user_list = []
    for index, row in best_posts.iterrows():
        post_id = row["id"]
        userId = get_username_from_post_metadata(post_id)    
        user_list.append(userId)
        
    users = list(dict.fromkeys(user_list))
    users_df = pd.DataFrame([[1, 2]],columns=["id","relevantMedia"])
    new_users = pd.DataFrame(columns=["id","relevantMedia"])
    for i in range(0,len(users)):
        matches = user_list.count(users[i])
        new_users = pd.DataFrame([[users[i], matches]],columns=["id","relevantMedia"])
        users_df = users_df.append(new_users)
    users_df = users_df.sort_values(by=['relevantMedia'], ascending = False)
    users_df = users_df.reset_index(drop=True)
    
    users_data = pd.DataFrame()
    for index, row in users_df.iterrows():
        userId = row["id"]
        relevantMedia = row["relevantMedia"]
        user_data = get_user_data(userId)
        new_user_data = pd.DataFrame.from_dict(user_data, orient='index')
        new_user_data = new_user_data.transpose()
        new_user_data = new_user_data.assign(relevantMedia=relevantMedia)
        users_data = users_data.append(new_user_data)
        
    users_data = users_data.reset_index(drop=True)
    users_data = users_data[['id', 'username', 'relevantMedia','followerCount','mediaCount']]
    
    #Obtención del ránking
    print("RÁNKING DE USUARIOS CONSIDERADOS COMO 'LOS NUESTROS':")
    users_data = users_data.head(target_users)
    print(users_data)
    
    #Seguimiento de cuentas seleccionadas
    for index, row in users_data.iterrows():
        userId = row["id"]
        follow_user(userId, user_id)
    
    print("")    
    print("¡Se han seguido a las cuentas seleccionadas!")    
        
    end = time.time()
    print("")
    print("Tiempo total de ejecución: "+str(round(end-start, 2))+" segundos")

#### Otras funciones:

In [3]:
def get_hashtags_list(keyword):
    with open('hashtags.json') as json_data:
        hashtags_json = json.load(json_data)
        hashtags = hashtags_json[keyword]
        json_data.close()
        return hashtags

In [4]:
def get_hashtag_id(hashtag, user_id):
    r = requests.get('http://hackathon.ocupa2.com/instagram/ig_hashtag_search?q='+hashtag+'&user_id'+user_id)
    hashtag_json = json.loads(r.text)
    hashtag_id = hashtag_json['id']
    return hashtag_id

In [5]:
def get_media_posts(hashtag, user_id, mediaType):
    hashtag_id = get_hashtag_id(hashtag, user_id)
    endpoint_media = 'http://hackathon.ocupa2.com/instagram/'+str(hashtag_id)+'/'+mediaType+'_media?user_id='+user_id
    r = requests.get(endpoint_media)
    posts_json = json.loads(r.text)
    posts_df = pd.DataFrame.from_dict(posts_json['data'])
    best_posts = posts_df.sort_values(by=['commentsCount', 'likeCount'], ascending = False).head(30)
    return best_posts

In [6]:
def get_best_media_posts(hashtags_category, user_id, mediaType):
    hashtags_list = get_hashtags_list(hashtags_category)
    sum_df = pd.DataFrame()
    for i in range(0, len(hashtags_list)):
        next_df = get_media_posts(hashtags_list[i], user_id, mediaType)
        sum_df = sum_df.append(next_df)
    sum_df = sum_df.sort_values(by=['commentsCount', 'likeCount'], ascending = False)
    return sum_df

In [7]:
def get_username_from_post_metadata(post_id):
    endpoint_username = 'http://hackathon.ocupa2.com/instagram/media/'+str(post_id)+'?fields=username'
    r = requests.get(endpoint_username)
    username = r.json()
    return username[0]['userId']

In [8]:
def get_user_data(userId):
    endpoint = 'http://hackathon.ocupa2.com/instagram/'+str(userId)+'?fields=id,follower_count,media_count,username'
    r = requests.get(endpoint)
    user_data = r.json()
    return user_data[0]

In [9]:
def follow_user(userId, user_id):
    endpoint = 'http://hackathon.ocupa2.com/instagram/'+str(userId)+"/follow?user_id="+str(user_id)+"&action=follow"
    r = requests.get(endpoint)

## FUNCIONES PARA TWITTER

#### Función principal:

In [10]:
def main_twitter():
    start = time.time()    
    
    twitter_user = '10013'
    twitter_key = 'muktdqLj37mxQgEwkRa6'
    hashtags_category = 'hashtags_food'
    target_users = 20
    tweets_list = get_tweets_list(hashtags_category)
    
    #Crear DataFrame con los metadatos relevantes:
    metadata_df = pd.DataFrame()
    for i in range(0,len(tweets_list)):
        dict_metadata = get_metadata_of_tweet(tweets_list[i])
        new_metadata = pd.DataFrame.from_dict(dict_metadata, orient='index')
        new_metadata = new_metadata.transpose()
        metadata_df = metadata_df.append(new_metadata)
    metadata_df = metadata_df.drop(['idStr'], axis=1)    

    #Dar RT y like a las publicaciones más relevantes:
    #TODO
    
    #Agrupar por usuarios los comentarios, RTs y likes de publicaciones relevantes:
    metadata_df = metadata_df.drop(['id'], axis=1)
    metadata_df = metadata_df.reset_index(drop=True)
    
    #Obtención del ránking
    ranking_df = get_user_ranking(metadata_df)
    ranking_df = ranking_df.head(target_users)
    ranking_df = ranking_df.astype('int64')
    ranking_df = ranking_df.reset_index(drop=True)
    ranking_df = ranking_df[['user', 'engagementRate', 'replyCount', 'retweetCount', 'likeCount']]
    
    print("RÁNKING DE USUARIOS CONSIDERADOS COMO 'LOS NUESTROS':")
    print(ranking_df)

    #Hacer follow a los usuarios seleccionados:
    for index, row in ranking_df.iterrows():
        user = row["user"]
        follow_user(user, twitter_key)    
        
    print("")    
    print("¡Se han seguido a las cuentas seleccionadas!")
    
    
    end = time.time()
    print("")
    print("Tiempo total de ejecución: "+str(round(end-start, 2))+" segundos")    

#### Otras funciones:

In [11]:
def get_hashtags_list(keyword):
    with open('hashtags.json') as json_data:
        hashtags_json = json.load(json_data)
        hashtags = hashtags_json[keyword]
        json_data.close()
        return hashtags

In [12]:
def get_tweets_by_hashtag(hashtag):
    tweet_ids = []
    r = requests.get('http://hackathon.ocupa2.com/twitter/1.1/search/tweets.json?q='+hashtag)
    tweets_json = r.json()
    for i in range(0,len(tweets_json)):
        tweet_ids.append(tweets_json[i]['tweetId'])
    return tweet_ids

In [13]:
def get_tweets_list(hashtags_category):
    hashtags_list = get_hashtags_list(hashtags_category)
    tweets_list = []
    for i in range(0, len(hashtags_list)):
        tweets_ids = get_tweets_by_hashtag(hashtags_list[i])
        tweets_list = tweets_list + tweets_ids
    tweets_list = list(dict.fromkeys(tweets_ids))    
    return tweets_list

In [14]:
def get_metadata_of_tweet(tweet):
    r = requests.get('http://hackathon.ocupa2.com/twitter/1.1/statuses/retweets/'+str(tweet)+'.json')
    tweet_metadata = r.json()
    return tweet_metadata[0]

In [15]:
def get_user_ranking(metadata_df):
    ranking_df = metadata_df.groupby(["user"], level=0).sum()
    ranking_df['engagementRate'] = ranking_df.apply(lambda row: (row.likeCount + row.retweetCount + row.replyCount), axis=1)
    ranking_df = ranking_df.sort_values(by=['engagementRate'], ascending = False)
    return ranking_df

In [16]:
def follow_user(user, twitter_key):
    endpoint = 'http://hackathon.ocupa2.com/twitter/1.1/friendships/create.json?user_id='+str(user)+'&bearer='+twitter_key
    r = requests.get(endpoint)

## EJECUCIÓN DEL PROGRAMA:

In [17]:
main_instagram()

RÁNKING DE USUARIOS CONSIDERADOS COMO 'LOS NUESTROS':
    id                    username  relevantMedia followerCount mediaCount
0    4    Greyson GLOVER CERVANTES             17            79        478
1   28       Rebekah BAXTER MATHIS             16            75        418
2   21      Kinslee TAYLOR MCCLURE             14            66        364
3    7        Cynthia BUCKLEY PACE             14            92        484
4   10      Emmalyn CARRILLO SWEET             13            82        406
5    6       Zaniyah MELENDEZ FORD             13            65        298
6    2          Jaxen BARKER SOLIS             13            82        418
7   49  Guillermo JUSTICE CANTRELL             13            56        326
8   14      Alannah TILLMAN HARDIN             12            74        354
9   59     Armando GILES CHRISTIAN             11            48        382
10  13           Belen SWEET NOBLE             10            56        334
11   5             Dean NUNEZ OWEN            

In [18]:
main_twitter()

RÁNKING DE USUARIOS CONSIDERADOS COMO 'LOS NUESTROS':
    user  engagementRate  replyCount  retweetCount  likeCount
0      2              15           1             7          7
1     89              15           3             6          6
2     18              15           4             5          6
3     67              15           2             6          7
4     16              14           3             6          5
5     12              13           2             5          6
6     56              13           3             3          7
7     69              13           3             3          7
8     82              12           3             3          6
9     74              12           2             3          7
10    14              12           2             4          6
11    28              12           2             4          6
12    29              12           2             4          6
13     1              11           2             3          6
14    86        

### POSIBLES MEJORAS A FUTURO

- Crear un limitador de consultas para no superar los límites impuestos por Twitter e Instagram
- Diseño de una interfaz gráfica para consultar métricas